# Case Study

Jack Dean, Charles Dotson, and Rodrigo Petricioli  
ISE 447 - Financial Optimization  
Lehigh Univerisity

In [1]:
'''
Package Imports
'''
import pandas as pd
import numpy as np
import pulp
from pulp import *
import datetime
import matplotlib.pyplot as plt

import FinOpsCodeDeck as finops

from IPython.display import Markdown as md
%matplotlib widget

## Provided Data

A municipality sends the following liability stream (in millions of dollars)

| Date Due | Amount |  
| -- | -- |
| 12/15/2022 | 11 |  
|  6/15/2023 |  9 |  
| 12/15/2023 |  8 |  
|  6/15/2024 |  7 |  
| 12/15/2024 |  9 |  
|  6/15/2025 | 10 |  
| 12/15/2025 |  9 |  
|  6/15/2026 | 12 |  
| 12/15/2026 |  9 |  
|  6/15/2027 |  6 |  
| 12/15/2027 |  5 |  
|  6/15/2028 |  7 |  
| 12/15/2028 |  9 |  
|  6/15/2029 |  7 |  
| 12/15/2029 |  8 |  
|  6/15/2030 |  7 |  

## Problem 1

Determine the current term structure of treasury rates (see textbook Section 3.4 or other resources that you can find), and find the present value, duration, and convexity of the stream of liabilities. Please explain the main steps followed in your calculations.  Use real world data.

In [2]:
'''
Data Import for Current Term Structure
---
Imports all active treasury bonds data, time indexes them by year
'''
term_structure_df = pd.read_excel('Table.xlsx', sheet_name='Bills Bonds and Notes')
term_structure_df['px_last'] = [i if i>=5 else 100 - i for i in term_structure_df['ASKED'].to_list()]
term_structure_df['ttm'] = [(i - datetime.datetime.now())/datetime.timedelta(days=365) for i in term_structure_df['MATURITY']]

In [3]:
'''
Bootstrap yield curve
---
begins with zero-coupon bonds to payout (ttm < 0.5 yrs) & calculates yield
moves to coupon bonds and uses calculated yields to bootstrap further
sorts all bonds into data frame indexed by ttm (by 100th of a year)
NOTE: Averages yields for the same time period
NOTE: assumes yield of period prior if yield for desired period does not exist
'''

'''short term rates'''
mats = []
round_to = 2
for bond_tenor in term_structure_df[term_structure_df['ttm'] <= 0.5].index:
    bond = term_structure_df.loc[bond_tenor]
    cpn = bond['COUPON']/2
    ttm = bond['ttm']
    px = bond['px_last']
    mats.append([np.round(ttm,round_to),np.log((100 + cpn) / bond['px_last']) / bond['ttm']])
rates = pd.DataFrame(mats, columns=['ttm','rate']).set_index('ttm').groupby('ttm').mean()

'''longer term rates'''
for bond_tenor in term_structure_df[term_structure_df['ttm']>=0.5].index:
    bond = term_structure_df.loc[bond_tenor]
    px = bond['px_last']
    ttm = bond['ttm']
    cpn = bond['COUPON']/2
    pmts = int(np.ceil(ttm * 2))
    cfs = [cpn if i+1<pmts else 100 + cpn for i in range(pmts)]
    cfs_idx = [np.round(ttm-i*0.5, round_to) for i in reversed(range(pmts))]
    known_rates = [rates[:cfs_idx[i]].iloc[-1,0] for i in range(pmts-1)]
    val = px - sum([cpn * np.exp((-1) * known_rates[i] * cfs_idx[i]) for i in range(pmts-1)])
    yld = (-1) * (np.log(val / (100+cpn)) / cfs_idx[pmts-1])
    add_df = pd.DataFrame([np.round(ttm, round_to), yld], index=['ttm','rate']).transpose().set_index('ttm')
    rates = pd.concat([rates,add_df],ignore_index=False)
    rates = rates.groupby('ttm').mean()

Text(0, 0.5, 'Yield in Percentage Points')

In [4]:
'''
plot yield curve
---
plots yield curve in percentage points
'''
plt.plot(rates[0.5:] * 100)
plt.title('Bootstrapped Yield Curve (rates per annum)')
plt.xlabel('Tenor')
plt.ylabel('Yield in Percentage Points')

In [5]:
data_prompt = pd.read_excel('Table.xlsx', sheet_name='PromptUse', index_col = 'DateDue')
data_prompt['ttm'] = np.round((data_prompt.index - datetime.datetime.now()) / datetime.timedelta(days=365), round_to)

r = []
for ttm in data_prompt['ttm']:
    r.append(rates[:ttm].iloc[-1,0])
data_prompt['rates'] = r

npv = sum([data_prompt.iloc[i,0]*np.exp((-1) * data_prompt.iloc[i,1] * data_prompt.iloc[i,2]) for i in range(len(data_prompt))])
dur = sum([data_prompt.iloc[i,1]*data_prompt.iloc[i,0]*np.exp((-1) * data_prompt.iloc[i,1] * data_prompt.iloc[i,2]) for i in range(len(data_prompt))])
con = sum([data_prompt.iloc[i,1]*(data_prompt.iloc[i,1]+1)*data_prompt.iloc[i,0]*np.exp((-1) * (data_prompt.iloc[i,1] + 2 ) * data_prompt.iloc[i,2]) for i in range(len(data_prompt))])

print('The Net Present Value of the Liabilities is ${:.2f} million dollars'.format(npv/1000))
print('The Macauley Duration of the Liability stream is {:.2f} years'.format(dur/npv))
print('The Convexity of the Liability stream is {:.2f}'.format(con/npv))

The Net Present Value of the Liabilities is $116.94 million dollars
The Macauley Duration of the Liability stream is 3.97 years
The Convexity of the Liability stream is 23.46


## Problem 2

Identify *at least* 30 fixed-income assets that are suitable to construct a dedicated bond portfolio for the municipality liabilities that you have been given. Use assets that are considered risk-free; for example, US government non-callable treasury bonds, treasury bills, or treasury notes. Display in an appropriate table the main characteristics of the bonds you choose. Namely, prices, coupon rates, maturity dates, face value).

In [14]:
'''
code block
'''

ref_data = []
for bond in term_structure_df.index:
    ref_data.append('T ' + str(term_structure_df.iloc[bond,1]) + ' ' + term_structure_df.iloc[bond,0].strftime('%m/%d/%y'))
term_structure_df['ref_data'] = ref_data
bonds_clean = term_structure_df[['ref_data', 'px_last', 'ASKED YIELD']]
bonds_clean.columns = ['Bond', 'Price', 'Yield']
bonds_clean = bonds_clean.set_index('Bond')
md(
    '''
    <center>  
    <br>
    {}  
    </center>
    '''.format(bonds_clean.to_markdown(colalign = ("right",)))
)


    <center>  
    <br>
    |             Bond |   Price |   Yield |
|-----------------:|--------:|--------:|
|   T 0.0 05/05/22 |  99.942 |  0.058  |
|   T 0.0 05/10/22 |  99.925 |  0.076  |
|   T 0.0 05/12/22 |  99.857 |  0.145  |
|  T 1.75 05/15/22 | 100.024 | -0.6005 |
| T 2.125 05/15/22 | 100.03  | -0.6944 |
|   T 0.0 05/17/22 |  99.815 |  0.188  |
|   T 0.0 05/19/22 |  99.762 |  0.241  |
|   T 0.0 05/24/22 |  99.68  |  0.325  |
|   T 0.0 05/26/22 |  99.675 |  0.33   |
|   T 0.0 05/31/22 |  99.63  |  0.375  |
| T 0.125 05/31/22 | 100     |  0.125  |
|  T 1.75 05/31/22 | 100.04  |  0.124  |
| T 1.875 05/31/22 | 100.034 |  0.45   |
|   T 0.0 06/02/22 |  99.665 |  0.34   |
|   T 0.0 06/07/22 |  99.597 |  0.408  |
|   T 0.0 06/09/22 |  99.595 |  0.411  |
|   T 0.0 06/14/22 |  99.577 |  0.429  |
|  T 1.75 06/15/22 | 100.046 |  0.49   |
|   T 0.0 06/16/22 |  99.582 |  0.424  |
|   T 0.0 06/21/22 |  99.48  |  0.528  |
|   T 0.0 06/23/22 |  99.5   |  0.507  |
|   T 0.0 06/28/22 |  99.337 |  0.672  |
|   T 0.0 06/30/22 |  99.35  |  0.66   |
| T 0.125 06/30/22 |  99.292 |  0.662  |
|  T 1.75 06/30/22 | 100.056 |  0.624  |
| T 2.125 06/30/22 | 100.076 |  0.608  |
|   T 0.0 07/07/22 |  99.29  |  0.721  |
|   T 0.0 07/14/22 |  99.262 |  0.749  |
|  T 1.75 07/15/22 | 100.062 |  0.777  |
|   T 0.0 07/21/22 |  99.232 |  0.78   |
|   T 0.0 07/28/22 |  99.212 |  0.8    |
| T 0.125 07/31/22 |  99.264 |  0.826  |
| T 1.875 07/31/22 | 100.082 |  0.821  |
|   T 2.0 07/31/22 | 100.092 |  0.819  |
|   T 0.0 08/04/22 |  99.112 |  0.902  |
|   T 0.0 08/11/22 |  99.055 |  0.961  |
|   T 1.5 08/15/22 | 100.05  |  0.953  |
| T 1.625 08/15/22 | 100.062 |  0.941  |
|  T 7.25 08/15/22 | 101.244 |  1.07   |
|   T 0.0 08/18/22 |  99.047 |  0.969  |
|   T 0.0 08/25/22 |  98.972 |  1.045  |
| T 0.125 08/31/22 |  99.222 |  1.063  |
| T 1.625 08/31/22 | 100.054 |  1.094  |
| T 1.875 08/31/22 | 100.082 |  1.079  |
|   T 0.0 09/01/22 |  98.927 |  1.091  |
|   T 0.0 09/08/22 |  98.93  |  1.089  |
|   T 0.0 09/15/22 |  98.912 |  1.107  |
|   T 1.5 09/15/22 | 100.042 |  1.135  |
|   T 0.0 09/22/22 |  98.927 |  1.092  |
|   T 0.0 09/29/22 |  98.89  |  1.131  |
| T 0.125 09/30/22 |  99.176 |  1.218  |
|  T 1.75 09/30/22 | 100.064 |  1.251  |
| T 1.875 09/30/22 | 100.082 |  1.241  |
|   T 0.0 10/06/22 |  98.85  |  1.172  |
|   T 0.0 10/13/22 |  98.742 |  1.282  |
| T 1.375 10/15/22 | 100.004 |  1.34   |
|   T 0.0 10/20/22 |  98.72  |  1.306  |
|   T 0.0 10/27/22 |  98.625 |  1.404  |
| T 0.125 10/31/22 |  99.12  |  1.405  |
| T 1.875 10/31/22 | 100.072 |  1.411  |
|   T 2.0 10/31/22 | 100.094 |  1.392  |
|   T 0.0 11/03/22 |  98.637 |  1.391  |
| T 1.625 11/15/22 | 100.022 |  1.492  |
| T 7.625 11/15/22 | 103.06  |  1.595  |
| T 0.125 11/30/22 |  99.06  |  1.545  |
|   T 2.0 11/30/22 | 100.082 |  1.549  |
|   T 0.0 12/01/22 |  98.607 |  1.424  |
| T 1.625 12/15/22 | 100.004 |  1.599  |
|   T 0.0 12/29/22 |  98.54  |  1.495  |
| T 0.137 12/31/22 |  98.306 |  1.714  |
| T 2.125 12/31/22 | 100.09  |  1.693  |
|   T 1.5 01/15/23 |  99.266 |  1.735  |
|   T 0.0 01/26/23 |  98.447 |  1.593  |
| T 0.125 01/31/23 |  98.244 |  1.798  |
|  T 1.75 01/31/23 |  99.31  |  1.791  |
| T 2.375 01/31/23 | 100.132 |  1.812  |
| T 1.375 02/15/23 |  99.204 |  1.836  |
|   T 2.0 02/15/23 | 100.036 |  1.848  |
| T 7.125 02/15/23 | 104.036 |  1.829  |
|   T 0.0 02/23/23 |  98.3   |  1.748  |
| T 0.125 02/28/23 |  98.166 |  1.935  |
|   T 1.5 02/28/23 |  99.21  |  1.92   |
| T 2.625 02/28/23 | 100.176 |  1.943  |
|   T 0.5 03/15/23 |  98.242 |  1.952  |
|   T 0.0 03/23/23 |  98.235 |  1.823  |
| T 0.125 03/31/23 |  98.094 |  2.024  |
|   T 1.5 03/31/23 |  99.174 |  2.005  |
|   T 2.5 03/31/23 | 100.126 |  2.055  |
|  T 0.25 04/15/23 |  98.084 |  2.102  |
|   T 0.0 04/20/23 |  98.017 |  2.05   |
| T 0.125 04/30/23 |  98.006 |  2.15   |
| T 1.625 04/30/23 |  99.154 |  2.153  |
|  T 2.75 04/30/23 | 100.186 |  2.15   |
| T 0.125 05/15/23 |  97.292 |  2.178  |
|  T 1.75 05/15/23 |  99.182 |  2.173  |
| T 0.125 05/31/23 |  97.256 |  2.199  |
| T 1.625 05/31/23 |  99.116 |  2.222  |
|  T 2.75 05/31/23 | 100.18  |  2.218  |
|  T 0.25 06/15/23 |  97.262 |  2.235  |
| T 0.125 06/30/23 |  97.174 |  2.279  |
| T 1.375 06/30/23 |  98.31  |  2.28   |
| T 2.625 06/30/23 | 100.124 |  2.281  |
| T 0.125 07/15/23 |  97.122 |  2.347  |
| T 0.125 07/31/23 |  97.074 |  2.391  |
|  T 1.25 07/31/23 |  98.204 |  2.363  |
|  T 2.75 07/31/23 | 100.14  |  2.39   |
| T 0.125 08/15/23 |  97.04  |  2.407  |
|   T 2.5 08/15/23 | 100.046 |  2.381  |
|  T 6.25 08/15/23 | 104.292 |  2.349  |
| T 0.125 08/31/23 |  96.306 |  2.469  |
| T 1.375 08/31/23 |  98.2   |  2.434  |
|  T 2.75 08/31/23 | 100.126 |  2.441  |
| T 0.125 09/15/23 |  96.284 |  2.452  |
|  T 0.25 09/30/23 |  96.302 |  2.468  |
| T 1.375 09/30/23 |  98.164 |  2.452  |
| T 2.875 09/30/23 | 100.17  |  2.488  |
| T 0.125 10/15/23 |  96.202 |  2.503  |
| T 0.375 10/31/23 |  96.272 |  2.539  |
| T 1.625 10/31/23 |  98.226 |  2.511  |
| T 2.875 10/31/23 | 100.154 |  2.542  |
|  T 0.25 11/15/23 |  96.174 |  2.561  |
|  T 2.75 11/15/23 | 100.096 |  2.546  |
|   T 0.5 11/30/23 |  96.25  |  2.596  |
| T 2.125 11/30/23 |  99.106 |  2.557  |
| T 2.875 11/30/23 | 100.144 |  2.579  |
| T 0.125 12/15/23 |  96.034 |  2.596  |
|  T 0.75 12/31/23 |  96.302 |  2.642  |
|  T 2.25 12/31/23 |  99.134 |  2.607  |
| T 2.625 12/31/23 | 100.014 |  2.595  |
| T 0.125 01/15/24 |  95.272 |  2.634  |
| T 0.875 01/31/24 |  96.31  |  2.663  |
|   T 2.5 01/31/24 |  99.234 |  2.656  |
| T 0.125 02/15/24 |  95.194 |  2.657  |
|  T 2.75 02/15/24 | 100.05  |  2.659  |
|   T 1.5 02/29/24 |  97.294 |  2.673  |
| T 2.125 02/29/24 |  99.012 |  2.667  |
| T 2.375 02/29/24 |  99.152 |  2.67   |
|  T 0.25 03/15/24 |  95.184 |  2.695  |
| T 2.125 03/31/24 |  98.294 |  2.708  |
|  T 2.25 03/31/24 |  99.046 |  2.71   |
| T 0.375 04/15/24 |  95.184 |  2.718  |
|   T 2.0 04/30/24 |  98.194 |  2.722  |
|  T 2.25 04/30/24 |  99.032 |  2.716  |
|   T 2.5 04/30/24 |  99.184 |  2.719  |
|  T 0.25 05/15/24 |  95.032 |  2.744  |
|   T 2.5 05/15/24 |  99.154 |  2.762  |
|   T 2.0 05/31/24 |  98.16  |  2.748  |
|  T 0.25 06/15/24 |  94.264 |  2.781  |
|  T 1.75 06/30/24 |  97.276 |  2.773  |
|   T 2.0 06/30/24 |  98.11  |  2.795  |
| T 0.375 07/15/24 |  94.266 |  2.81   |
|  T 1.75 07/31/24 |  97.226 |  2.808  |
| T 2.125 07/31/24 |  98.162 |  2.814  |
| T 0.375 08/15/24 |  94.182 |  2.844  |
| T 2.375 08/15/24 |  98.296 |  2.861  |
|  T 1.25 08/31/24 |  96.136 |  2.847  |
| T 1.875 08/31/24 |  97.256 |  2.857  |
| T 0.375 09/15/24 |  94.102 |  2.874  |
|   T 1.5 09/30/24 |  96.272 |  2.861  |
| T 2.125 09/30/24 |  98.094 |  2.861  |
| T 0.625 10/15/24 |  94.226 |  2.866  |
|   T 1.5 10/31/24 |  96.216 |  2.891  |
|  T 2.25 10/31/24 |  98.146 |  2.895  |
|  T 0.75 11/15/24 |  94.244 |  2.907  |
|  T 2.25 11/15/24 |  98.132 |  2.904  |
|   T 7.5 11/15/24 | 111.11  |  2.827  |
|   T 1.5 11/30/24 |  96.176 |  2.897  |
| T 2.125 11/30/24 |  98.026 |  2.901  |
|   T 1.0 12/15/24 |  95.08  |  2.894  |
|  T 1.75 12/31/24 |  97.012 |  2.914  |
|  T 2.25 12/31/24 |  98.102 |  2.91   |
| T 1.125 01/15/25 |  95.116 |  2.92   |
| T 1.375 01/31/25 |  95.304 |  2.919  |
|   T 2.5 01/31/25 |  98.294 |  2.911  |
|   T 1.5 02/15/25 |  96.052 |  2.941  |
|   T 2.0 02/15/25 |  97.174 |  2.922  |
| T 7.625 02/15/25 | 112.216 |  2.857  |
| T 1.125 02/28/25 |  95.052 |  2.92   |
|  T 2.75 02/28/25 |  99.17  |  2.923  |
|  T 1.75 03/15/25 |  96.246 |  2.931  |
|   T 0.5 03/31/25 |  93.086 |  2.928  |
| T 2.625 03/31/25 |  99.044 |  2.935  |
| T 2.625 04/15/25 |  99.054 |  2.92   |
| T 0.375 04/30/25 |  92.224 |  2.941  |
| T 2.875 04/30/25 |  99.262 |  2.938  |
| T 2.125 05/15/25 |  97.2   |  2.949  |
|  T 0.25 05/31/25 |  92.032 |  2.954  |
| T 2.875 05/31/25 |  99.252 |  2.947  |
|  T 0.25 06/30/25 |  91.276 |  2.965  |
|  T 2.75 06/30/25 |  99.124 |  2.953  |
|  T 0.25 07/31/25 |  91.2   |  2.976  |
| T 2.875 07/31/25 |  99.234 |  2.961  |
|   T 2.0 08/15/25 |  96.306 |  2.977  |
| T 6.875 08/15/25 | 112.102 |  2.916  |
|  T 0.25 08/31/25 |  91.132 |  2.981  |
|  T 2.75 08/31/25 |  99.094 |  2.973  |
|  T 0.25 09/30/25 |  91.062 |  2.985  |
|   T 3.0 09/30/25 | 100.03  |  2.97   |
|  T 0.25 10/31/25 |  91.002 |  2.981  |
|   T 3.0 10/31/25 | 100.026 |  2.974  |
|  T 2.25 11/15/25 |  97.174 |  2.987  |
| T 0.375 11/30/25 |  91.054 |  2.996  |
| T 2.875 11/30/25 |  99.196 |  2.988  |
| T 0.375 12/31/25 |  90.316 |  2.991  |
| T 2.625 12/31/25 |  98.244 |  2.983  |
| T 0.375 01/31/26 |  90.24  |  3.004  |
| T 2.625 01/31/26 |  98.224 |  2.993  |
| T 1.625 02/15/26 |  95.026 |  3.007  |
|   T 6.0 02/15/26 | 110.244 |  2.972  |
|   T 0.5 02/28/26 |  91.002 |  3.005  |
|   T 2.5 02/28/26 |  98.066 |  2.998  |
|  T 0.75 03/31/26 |  91.232 |  3.009  |
|  T 2.25 03/31/26 |  97.076 |  3.003  |
|  T 0.75 04/30/26 |  91.182 |  3.007  |
| T 2.375 04/30/26 |  97.214 |  2.998  |
| T 1.625 05/15/26 |  94.246 |  3.011  |
|  T 0.75 05/31/26 |  91.116 |  3.016  |
| T 2.125 05/31/26 |  96.202 |  3.009  |
| T 0.875 06/30/26 |  91.214 |  3.02   |
| T 1.875 06/30/26 |  95.204 |  2.997  |
| T 0.625 07/31/26 |  90.162 |  3.024  |
| T 1.875 07/31/26 |  95.18  |  2.995  |
|   T 1.5 08/15/26 |  93.29  |  3.026  |
|  T 6.75 08/15/26 | 114.296 |  3.011  |
|  T 0.75 08/31/26 |  90.272 |  3.022  |
| T 1.375 08/31/26 |  93.12  |  3.02   |
| T 0.875 09/30/26 |  91.06  |  3.025  |
| T 1.625 09/30/26 |  94.082 |  3.026  |
| T 1.125 10/31/26 |  92.022 |  3.026  |
| T 1.625 10/31/26 |  94.05  |  3.026  |
|   T 2.0 11/15/26 |  95.196 |  3.042  |
|   T 6.5 11/15/26 | 114.194 |  3.026  |
|  T 1.25 11/30/26 |  92.164 |  3.013  |
| T 1.625 11/30/26 |  94.024 |  3.02   |
|  T 1.25 12/31/26 |  92.112 |  3.022  |
|  T 1.75 12/31/26 |  94.166 |  3.018  |
|   T 1.5 01/31/27 |  93.096 |  3.025  |
|  T 2.25 02/15/27 |  96.172 |  3.032  |
| T 6.625 02/15/27 | 115.28  |  3.036  |
| T 1.125 02/28/27 |  91.174 |  3.02   |
| T 1.875 02/28/27 |  94.302 |  3.008  |
| T 0.625 03/31/27 |  89.044 |  3.021  |
|   T 2.5 03/31/27 |  97.212 |  3.015  |
|   T 0.5 04/30/27 |  88.124 |  3.023  |
|  T 2.75 04/30/27 |  98.26  |  3.008  |
| T 2.375 05/15/27 |  97     |  3.022  |
|   T 0.5 05/31/27 |  88.036 |  3.044  |
|   T 0.5 06/30/27 |  87.304 |  3.04   |
| T 0.375 07/31/27 |  87.084 |  3.018  |
|  T 2.25 08/15/27 |  96.042 |  3.047  |
| T 6.375 08/15/27 | 116.06  |  3.037  |
|   T 0.5 08/31/27 |  87.176 |  3.049  |
|   T 0.5 10/31/27 |  87.062 |  3.05   |
|  T 2.25 11/15/27 |  95.312 |  3.046  |
| T 6.125 11/15/27 | 115.18  |  3.047  |
| T 0.625 11/30/27 |  87.2   |  3.055  |
| T 0.625 12/31/27 |  87.142 |  3.057  |
|  T 0.75 01/31/28 |  87.29  |  3.061  |
|  T 2.75 02/15/28 |  98.126 |  3.054  |
| T 1.125 02/29/28 |  89.256 |  3.048  |
|  T 1.25 03/31/28 |  90.086 |  3.062  |
|  T 1.25 04/30/28 |  90.046 |  3.062  |
| T 2.875 05/15/28 |  98.304 |  3.066  |
|  T 1.25 05/31/28 |  90     |  3.066  |
|  T 1.25 06/30/28 |  89.272 |  3.071  |
|   T 1.0 07/31/28 |  88.096 |  3.072  |
| T 2.875 08/15/28 |  98.286 |  3.069  |
|   T 5.5 08/15/28 | 113.294 |  3.049  |
| T 1.125 08/31/28 |  88.284 |  3.07   |
|  T 1.25 09/30/28 |  89.142 |  3.077  |
| T 1.375 10/31/28 |  90.022 |  3.074  |
| T 3.125 11/15/28 | 100.12  |  3.061  |
|  T 5.25 11/15/28 | 112.316 |  3.042  |
|   T 1.5 11/30/28 |  90.226 |  3.07   |
| T 1.375 12/31/28 |  89.296 |  3.058  |
|  T 1.75 01/31/29 |  92.024 |  3.058  |
| T 2.625 02/15/29 |  97.13  |  3.051  |
|  T 5.25 02/15/29 | 113.144 |  3.041  |
| T 1.875 02/28/29 |  92.264 |  3.046  |
| T 2.375 03/31/29 |  95.256 |  3.053  |
| T 2.375 05/15/29 |  95.252 |  3.045  |
| T 1.625 08/15/29 |  90.294 |  3.022  |
| T 6.125 08/15/29 | 120.1   |  3      |
|  T 1.75 11/15/29 |  91.182 |  3.009  |
|   T 1.5 02/15/30 |  89.186 |  3.01   |
| T 0.625 05/15/30 |  83.024 |  3.012  |
|  T 6.25 05/15/30 | 123.066 |  2.98   |
| T 0.625 08/15/30 |  82.204 |  3.007  |
| T 0.875 11/15/30 |  84.036 |  2.998  |
| T 1.125 02/15/31 |  85.21  |  2.993  |
| T 5.375 02/15/31 | 118.146 |  2.972  |
| T 1.625 05/15/31 |  89.072 |  2.995  |
|  T 1.25 08/15/31 |  85.302 |  2.995  |
| T 1.375 11/15/31 |  86.204 |  2.996  |
| T 1.875 02/15/32 |  90.182 |  2.994  |
|   T 4.5 02/15/36 | 118.086 |  2.884  |
|  T 4.75 02/15/37 | 121.08  |  2.964  |
|   T 5.0 05/15/37 | 124.084 |  2.985  |
| T 4.375 02/15/38 | 117.002 |  3.013  |
|   T 4.5 05/15/38 | 118.17  |  3.032  |
|   T 3.5 02/15/39 | 105.092 |  3.094  |
|  T 4.25 05/15/39 | 115.082 |  3.091  |
|   T 4.5 08/15/39 | 118.214 |  3.097  |
| T 4.375 11/15/39 | 116.222 |  3.13   |
| T 4.625 02/15/40 | 120.122 |  3.123  |
| T 1.125 05/15/40 |  70.312 |  3.268  |
| T 4.375 05/15/40 | 116.222 |  3.154  |
| T 1.125 08/15/40 |  70.182 |  3.277  |
| T 3.875 08/15/40 | 109.096 |  3.199  |
| T 1.375 11/15/40 |  73.216 |  3.282  |
|  T 4.25 11/15/40 | 114.124 |  3.213  |
| T 1.875 02/15/41 |  80.144 |  3.276  |
|  T 4.75 02/15/41 | 121.282 |  3.192  |
|  T 2.25 05/15/41 |  85.126 |  3.288  |
| T 4.375 05/15/41 | 116.046 |  3.232  |
|  T 1.75 08/15/41 |  78.01  |  3.298  |
|  T 3.75 08/15/41 | 107.026 |  3.252  |
| T 3.125 11/15/41 |  97.292 |  3.27   |
|   T 2.0 11/30/41 |  81.16  |  3.292  |
| T 2.375 02/15/42 |  87.004 |  3.271  |
| T 3.125 02/15/42 |  97.302 |  3.267  |
|   T 3.0 05/15/42 |  95.304 |  3.277  |
|  T 2.75 08/15/42 |  91.276 |  3.303  |
|  T 2.75 11/15/42 |  91.234 |  3.308  |
| T 3.125 02/15/43 |  97.12  |  3.3    |
| T 2.875 05/15/43 |  93.144 |  3.31   |
| T 3.625 08/15/43 | 105.036 |  3.289  |
|  T 3.75 11/15/43 | 107.03  |  3.288  |
| T 3.625 02/15/44 | 105.06  |  3.289  |
| T 3.375 05/15/44 | 101.07  |  3.297  |
| T 3.125 08/15/44 |  97.076 |  3.301  |
|   T 3.0 11/15/44 |  95.06  |  3.305  |
|   T 2.5 02/15/45 |  87.06  |  3.305  |
|   T 3.0 05/15/45 |  95.094 |  3.293  |
| T 2.875 08/15/45 |  93.11  |  3.286  |
|   T 3.0 11/15/45 |  95.174 |  3.273  |
|   T 2.5 02/15/46 |  87.094 |  3.273  |
|   T 2.5 05/15/46 |  87.082 |  3.27   |
|  T 2.25 08/15/46 |  83.02  |  3.265  |
| T 2.875 11/15/46 |  93.25  |  3.245  |
|   T 3.0 02/15/47 |  95.294 |  3.24   |
|   T 3.0 05/15/47 |  96.01  |  3.232  |
|  T 2.75 08/15/47 |  91.3   |  3.218  |
|  T 2.75 11/15/47 |  91.312 |  3.213  |
|   T 3.0 02/15/48 |  96.224 |  3.188  |
| T 3.125 05/15/48 |  99.062 |  3.171  |
|   T 3.0 08/15/48 |  96.244 |  3.182  |
| T 3.375 11/15/48 | 104.03  |  3.146  |
|   T 3.0 02/15/49 |  97.174 |  3.136  |
| T 2.875 05/15/49 |  95.104 |  3.132  |
|  T 2.25 08/15/49 |  83.302 |  3.129  |
| T 2.375 11/15/49 |  86.126 |  3.114  |
|   T 2.0 02/15/50 |  79.064 |  3.125  |
|  T 1.25 05/15/50 |  65.1   |  3.114  |
| T 1.375 08/15/50 |  67.142 |  3.115  |
| T 1.625 11/15/50 |  72.016 |  3.109  |
| T 1.875 02/15/51 |  76.28  |  3.094  |
| T 2.375 05/15/51 |  86.122 |  3.089  |
|   T 2.0 08/15/51 |  79.076 |  3.081  |
| T 1.875 11/15/51 |  76.28  |  3.072  |
|  T 2.25 02/15/52 |  84.072 |  3.061  |  
    </center>
    

## Problem 3

Formulate a linear programming model to find the lowest cost bond dedicated portfolio that covers the stream of liabilities. To eliminate the possibility of any interest risk, assume that a 0% reinvestment rate on cash balances carried out from one date to the next. Assume no short selling of bonds is allowed. What is the cost of your portfolio? How does this cost compares with the NPV of the liabilities? What is the composition of the portfolio?

In [7]:
'''
code block
'''

'\ncode block\n'

## Problem 4

Use the linear programming sensitivity analysis information to determine the term structure of interest rates implied by the optimal bond portfolio you found in the previous question. Use a plot to compare these rates with the current term structure of treasury rates you found in the first question.

In [8]:
'''
code block
'''

'\ncode block\n'

## Problem 5

Formulate a linear programming model to find the lowest cost bond immunized portfolio that matches the present value, duration, and convexity of a stream of liabilities. Assume that no short rates are allowed. What is the cost of your portfolio? How much would you save by using this immunization strategy instead of the dedication one? Is your portfolio immunized against non-parallel shifts in the term structure? Explain why or why not.

In [9]:
'''
code block
'''

'\ncode block\n'

## Problem 6

Combine a cash matching strategy (dedication) for the liabilities for the first three years and an immunization strategy based on matching present value, duration and convexity for the liabilities during the last five years. Compare the characteristics of the three bond portfolios you have obtained. Explain which one you think is the best one and why.

In [10]:
'''
code block
'''

'\ncode block\n'

## Problem 7

The municipality would like to make a second bid (find a different portfolio of bonds). What is your best dedicated portfolio of risk-free bonds you can create *if short sales are allowed*? Did you find arbitrage opportunities? Did you take into consideration the bid-ask spread of the bonds? How would you take them in consideration and what is the result? Did you set limits in the transaction amounts? Discuss the practical feasibility of your solutions.

In [11]:
'''
code block
'''

'\ncode block\n'

## Problem 8

Consider proposing a new portfolio of bonds using any additional consideration or change to the model that you see fit. For example, can you do something to make your portfolio of bonds immune to nonparallel changes in the term structure. Is there a better way to combine the techniques you used before. Explain clearly what you do and your results.

In [12]:
'''
code block
'''

'\ncode block\n'